In [1]:
import os
import geopandas as gpd

local_crs = 32636
example_data_path = "./my_data"

In [3]:
from blocksnet import GraphGenerator

blocks = gpd.read_parquet(os.path.join(example_data_path, 'blocks_cutter_result.parquet')).to_crs(local_crs)
# intermodal_graph = GraphGenerator(city_geometry=blocks, local_crs=local_crs).get_graph('intermodal')

In [4]:
df_blocks = gpd.GeoDataFrame(blocks, crs=local_crs)
df_blocks.head()

,geometry,id,landuse
0,"POLYGON ((303022.702 6675775.060, 303022.795 6...",0,no_dev_area
1,"POLYGON ((303012.608 6675801.562, 303012.586 6...",1,no_dev_area
2,"POLYGON ((303064.339 6675806.849, 303064.442 6...",2,no_dev_area
3,"POLYGON ((303428.643 6675829.284, 303425.776 6...",3,buildings
4,"POLYGON ((303429.370 6675800.664, 303429.236 6...",4,no_dev_area


In [5]:
houses = gpd.read_file(os.path.join(example_data_path, "house_points.geojson")).to_crs(local_crs)
df_houses = gpd.GeoDataFrame(houses, crs=local_crs)
df_houses.head()

,avg_m2_price_rent,mean_price_rent,P_clinic,P_hospitals,P_kindergarten,P_mfc,P_schools,accs_time,log_accs_time,Iq,P_avg,Idx,geometry
0,1190.476190,25000.0,1,1,1.0,0.0,1.0,40.8,1.61,0.55,0.8,0.675,POINT (349844.614 6646516.903)
1,537.634409,30000.0,1,1,1.0,0.0,1.0,83.0,1.92,0.37,0.8,0.585,POINT (349100.178 6657288.179)
2,708.573345,44666.0,1,1,1.0,0.0,1.0,10.0,1.00,0.50,0.8,0.650,POINT (342768.230 6637807.041)
3,538.461538,35000.0,1,1,1.0,0.0,1.0,10.0,1.00,0.61,0.8,0.705,POINT (342687.422 6637725.165)
4,2000.000000,32000.0,1,1,0.0,1.0,0.0,39.5,1.60,0.42,0.6,0.510,POINT (350044.641 6646061.322)


In [6]:
df_houses['Iq'].describe()

count    1505.000000
mean        0.405654
std         0.088457
min         0.300000
25%         0.340000
50%         0.380000
75%         0.490000
max         0.790000
Name: Iq, dtype: float64

In [7]:
df_houses = df_houses.loc[df_houses['Iq'] > 0.3]
df_houses['Iq'].describe()

count    1404.000000
mean        0.413255
std         0.086756
min         0.310000
25%         0.350000
50%         0.380000
75%         0.490000
max         0.790000
Name: Iq, dtype: float64

In [8]:
df_houses_pol = df_houses.copy()
df_houses_pol['geometry'] = df_houses['geometry'].apply(lambda x: x.buffer(0.001))  # Преобразование точек в небольшие полигоны
df_houses_pol.head()

,avg_m2_price_rent,mean_price_rent,P_clinic,P_hospitals,P_kindergarten,P_mfc,P_schools,accs_time,log_accs_time,Iq,P_avg,Idx,geometry
0,1190.476190,25000.0,1,1,1.0,0.0,1.0,40.8,1.61,0.55,0.8,0.675,"POLYGON ((349844.615 6646516.903, 349844.615 6..."
1,537.634409,30000.0,1,1,1.0,0.0,1.0,83.0,1.92,0.37,0.8,0.585,"POLYGON ((349100.179 6657288.179, 349100.179 6..."
2,708.573345,44666.0,1,1,1.0,0.0,1.0,10.0,1.00,0.50,0.8,0.650,"POLYGON ((342768.231 6637807.041, 342768.231 6..."
3,538.461538,35000.0,1,1,1.0,0.0,1.0,10.0,1.00,0.61,0.8,0.705,"POLYGON ((342687.423 6637725.165, 342687.423 6..."
4,2000.000000,32000.0,1,1,0.0,1.0,0.0,39.5,1.60,0.42,0.6,0.510,"POLYGON ((350044.642 6646061.322, 350044.642 6..."


In [9]:
# Выполнение пространственного объединения для определения, в каких кварталах находятся дома
overlay = gpd.overlay(df_blocks, df_houses_pol, how="intersection")

unique_blocks_ids = overlay["id"].unique()

df_blocks_with_houses = df_blocks[df_blocks["id"].isin(unique_blocks_ids)]
df_blocks_with_houses.head()

,geometry,id,landuse
2854,"POLYGON ((337030.970 6638333.827, 337031.347 6...",2854,selected_area
3326,"POLYGON ((339076.063 6636586.371, 339075.615 6...",3326,no_dev_area
3331,"POLYGON ((338883.944 6636688.475, 338884.040 6...",3331,buildings
3503,"POLYGON ((339697.103 6635459.892, 339690.468 6...",3503,no_dev_area
3504,"POLYGON ((340239.631 6635418.232, 340239.373 6...",3504,no_dev_area


In [10]:
df_blocks_with_houses.explore()